<a href="https://colab.research.google.com/github/alexandraizhevskaya/Psh-psh/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework

Привет! В этой домашнем задании ты научишься обучении модели BERT. На семинаре был разобран код модели, здесь же посмотрим на то, как надо обработать данные, чтобы на них модель могла учиться. 

Замечания по выполнению задания:

- Код внутри блока `<DON'T TOUCH THIS!>` используется для проверки задания, его нельзя трогать. 

- Внутри блока `<YOUR CODE>` может больше кода, чем там показано изначально.

- От задания требуется написания небольшого отчета в конце.


Для начала загрузи нужные библиотеки.

In [0]:
!pip install transformers catalyst

In [0]:
! pip install alchemy-catalyst

In [0]:
import os
import random
import sys
import urllib.request
import zipfile

import numpy as np
import pandas as pd
from tqdm import tqdm, trange

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, Dataset

import transformers

from catalyst.dl import SupervisedRunner
from catalyst.dl.callbacks import AccuracyCallback, SchedulerCallback, F1ScoreCallback
from catalyst.utils import set_global_seed, prepare_cudnn

Внизу идет технический код, который нужен для загрузки датасетов. Его можно уменьшить, выбрав только некоторые из них. Для того, что бы зачесть задание, надо выбрать не менее двух задач, для хотя бы одной из которых нужно использовать два предложения(ответ и вопрос, два предложения и прочее). Подробнее про датасеты [здесь](https://gluebenchmark.com/).

In [0]:
TASKS = ["CoLA", "SST", "MRPC", "QQP", "STS", "MNLI", "SNLI", "QNLI", "RTE", "WNLI"]
TASK2PATH = {
    "CoLA": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FCoLA.zip?alt=media&token=46d5e637-3411-4188-bc44-5809b5bfb5f4",
    "SST": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8",
    "MRPC": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc",
    "QQP": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQQP.zip?alt=media&token=700c6acf-160d-4d89-81d1-de4191d02cb5",
    "STS": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSTS-B.zip?alt=media&token=bddb94a7-8706-4e0d-a694-1109e12273b5",
    "MNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FMNLI.zip?alt=media&token=50329ea1-e339-40e2-809c-10c40afff3ce",
    "SNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSNLI.zip?alt=media&token=4afcfbb2-ff0c-4b2d-a09a-dbf07926f4df",
    "QNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQNLIv2.zip?alt=media&token=6fdcf570-0fc5-4631-8456-9505272d1601",
    "RTE": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FRTE.zip?alt=media&token=5efa7e85-a0bb-4f19-8ea2-9e1840f077fb",
    "WNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FWNLI.zip?alt=media&token=068ad0a0-ded7-4bd7-99a5-5e00222e0faf",
}

MRPC_TRAIN = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt"
MRPC_TEST = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt"

data_dir = "data/"
max_seq_length = 128

In [0]:
def download_and_extract(task, data_dir):
    print("Downloading and extracting %s..." % task)
    data_file = "%s.zip" % task
    urllib.request.urlretrieve(TASK2PATH[task], data_file)
    with zipfile.ZipFile(data_file) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(data_file)
    print("\tCompleted!")

def format_mrpc(data_dir, path_to_data):
    print("Processing MRPC...")
    mrpc_dir = os.path.join(data_dir, "MRPC")
    if not os.path.isdir(mrpc_dir):
        os.mkdir(mrpc_dir)
    if path_to_data:
        mrpc_train_file = os.path.join(path_to_data, "msr_paraphrase_train.txt")
        mrpc_test_file = os.path.join(path_to_data, "msr_paraphrase_test.txt")
    else:
        print("Local MRPC data not specified, downloading data from %s" % MRPC_TRAIN)
        mrpc_train_file = os.path.join(mrpc_dir, "msr_paraphrase_train.txt")
        mrpc_test_file = os.path.join(mrpc_dir, "msr_paraphrase_test.txt")
        urllib.request.urlretrieve(MRPC_TRAIN, mrpc_train_file)
        urllib.request.urlretrieve(MRPC_TEST, mrpc_test_file)
    assert os.path.isfile(mrpc_train_file), "Train data not found at %s" % mrpc_train_file
    assert os.path.isfile(mrpc_test_file), "Test data not found at %s" % mrpc_test_file
    urllib.request.urlretrieve(TASK2PATH["MRPC"], os.path.join(mrpc_dir, "dev_ids.tsv"))

    dev_ids = []
    with open(os.path.join(mrpc_dir, "dev_ids.tsv"), encoding="utf8") as ids_fh:
        for row in ids_fh:
            dev_ids.append(row.strip().split("\t"))

    with open(mrpc_train_file, encoding="utf8") as data_fh, open(
        os.path.join(mrpc_dir, "train.tsv"), "w", encoding="utf8"
    ) as train_fh, open(os.path.join(mrpc_dir, "dev.tsv"), "w", encoding="utf8") as dev_fh:
        header = data_fh.readline()
        train_fh.write(header)
        dev_fh.write(header)
        for row in data_fh:
            label, id1, id2, s1, s2 = row.strip().split("\t")
            if [id1, id2] in dev_ids:
                dev_fh.write("%s\t%s\t%s\t%s\t%s\n" % (label, id1, id2, s1, s2))
            else:
                train_fh.write("%s\t%s\t%s\t%s\t%s\n" % (label, id1, id2, s1, s2))

    with open(mrpc_test_file, encoding="utf8") as data_fh, open(
        os.path.join(mrpc_dir, "test.tsv"), "w", encoding="utf8"
    ) as test_fh:
        header = data_fh.readline()
        test_fh.write("index\t#1 ID\t#2 ID\t#1 String\t#2 String\n")
        for idx, row in enumerate(data_fh):
            label, id1, id2, s1, s2 = row.strip().split("\t")
            test_fh.write("%d\t%s\t%s\t%s\t%s\n" % (idx, id1, id2, s1, s2))
    print("\tCompleted!")

In [0]:
TASKS = ["QQP", "SST", "MRPC"] # Или можно просто сюда вписать те датасеты, которые ты выбрал.

for task in TASKS:
    if task == "MRPC":
        format_mrpc(data_dir, None)
    else:
        download_and_extract(task, data_dir)

Загрузи один из выбранных датасет с помощью Pandas(не обязательно через него, но так проще) и посмотри на него.

Загружаем SST-2 в качестве 1-го типа задач (где нужно одно предложение). Здесть обычная бинарная классификация на positive/negative. Смотрим на датасет, бьем его на треин и тест

In [0]:
# Вместо test-а возьмите valid, а valid сделай из train.

# <YOUR CODE>
train_pd = pd.read_csv('/content/data/SST-2/train.tsv', sep='\t')
test_pd = pd.read_csv('/content/data/SST-2/dev.tsv', sep='\t')
#test_pd = pd.read_csv('/content/data/SST-2/test.tsv', sep='\t')
# </YOUR CODE>

In [0]:
train_pd = train_pd[:int(len(train_pd)*0.97)]
val_pd = train_pd[int(len(train_pd)*0.97):]

In [0]:
train_pd.head()

In [0]:
val_pd.head()

In [0]:
test_pd.head()

`В качестве второго датасета берем QQP, где уже надо сравнить 2 предложения, чтобы определить, являются ли они дупликатами.

In [0]:
# Вместо test-а возьмите valid, а valid сделай из train.

# <YOUR CODE>
train_pd = pd.read_csv('/content/data/QQP/train.tsv', sep='\t', error_bad_lines=False)
test_pd = pd.read_csv('/content/data/QQP/dev.tsv', sep='\t', error_bad_lines=False)
#test_pd = pd.read_csv('/content/data/SST-2/test.tsv', sep='\t')
# </YOUR CODE>

b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.



In [0]:
train_pd.dropna(inplace=True)
test_pd.dropna(inplace=True)

In [0]:
train_pd = train_pd[:int(len(train_pd)*0.89)]
val_pd = train_pd[int(len(train_pd)*0.89):]

In [0]:
train_pd.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


Для начала рассмотрим важную часть обработки текста для трансфомера(и не только) – токенайзер.

В качестве примера токенайзера воспользуемся внутренним из библиотеки transformers, обученным для BERT-а. Посмотрим, что он умеет.

In [0]:
model_name = 'bert-base-uncased'

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

Посмотрим, как происходит токенизация предложения.

In [0]:
test_sentence = "Hide new secretions from the parental units."
print(tokenizer.tokenize(test_sentence))

Видно, что предложения разделяются не на слова, а подслова. Токены, которые надо объеденить в слова для получения "нормального" текста, выделены с помощью `##`. Посмотрим, как различаются коды токенов с этим символом и без него.

In [0]:
print(tokenizer.convert_tokens_to_ids(['ions']))
print(tokenizer.convert_tokens_to_ids(['##ions']))

Для токенизации предложений воспользуемся методом `encode`. Она принимает предложение как строку или список токенов**(!)**.

In [0]:
print(tokenizer.encode(test_sentence))

Добавились специальные токены впереди и сзади предложения. Посмотрим на весь список специальных токенов:

In [0]:
print(tokenizer.special_tokens_map)
print({i: j for i, j in zip(tokenizer.all_special_tokens, tokenizer.all_special_ids)})

Посмотрим, что ещё может делать токенайзер. Что требуется нам для обучения BERT-а: добавить паддинг, получить маску аттеншена и тип токенов. Попробуем сделать это самостоятельно и посмотрим, как это сделать с помощью токенайзера.

Выбери два предложения из обучающей выборки. Получи их токены с помощью метода `tokenize`. Объедени списки токенов так, чтобы модель могла различать, что они от разных предложений. 

(Подсказка: на семинаре была картинка с эмбеддингами. Она может подсказать, что надо изменить в токенах предложения) 

In [0]:
# <YOUR CODE>
s1, s2 = train_pd.sentence[1], train_pd.sentence[0]
tokenized_s1, tokenized_s2 = tokenizer.tokenize(s1), tokenizer.tokenize(s2)
s_union = tokenized_s1+['[SEP]']+tokenized_s2
# </YOUR CODE>

# <DON'T TOUCH THIS!>
assert tokenizer.encode(s_union) == tokenizer.encode(s1, s2), "Not equal"
# </DON'T TOUCH THIS!>

Теперь надо добавь нулей в полученный список чисел, чтобы они легко складывались в батчи.

In [0]:
# <YOUR CODE>
encoded_full = tokenizer.encode(s_union)+[0 for i in range((max_seq_length - len(s_union)-2))]

# <DON'T TOUCH THIS!>
encoded_correct = tokenizer.encode(s1, s2, max_length=max_seq_length, pad_to_max_length=True)
assert len(encoded_full) == len(encoded_correct), "Different length"
assert encoded_full == encoded_correct, "Not equal"
# </DON'T TOUCH THIS!>

В модель также надо кинуть маску для механизма внимания и тип предложения для каждого токена. Сделай их.

In [0]:
# <YOUR CODE>
token_type_ids = [0 for i in range(10)] + [1 for i in range(10)] + [0 for i in range(108)]
attention_mask = [1  if i!=0 else 0 for i in encoded_full]
# </YOUR CODE>

# <DON'T TOUCH THIS!>
encoded_plus = tokenizer.encode_plus(train_pd['sentence'][0], text_pair=train_pd['sentence'][1], max_length=max_seq_length, pad_to_max_length=True)
assert len(token_type_ids) == len(encoded_plus['token_type_ids']), "Different length in token_type_ids"
assert token_type_ids == encoded_plus['token_type_ids'], "Not equal token_type_ids"
assert len(attention_mask) == len(encoded_plus['attention_mask']), "Different length in attention_mask"
assert attention_mask == encoded_plus['attention_mask'], "Not equal attention_mask"
# </DON'T TOUCH THIS!>

Как видно из тестов, все нужные для обработки текста для BERT-а вещи может делать токенизатор из `transformers`. Но не все токенизаторы настолько функциональны. Их (почти)полный список:
- [Sentence Piece](https://github.com/google/sentencepiece/)
- [fastBPE](https://github.com/glample/fastBPE)
- [Hugging Face Tokenizers](https://github.com/huggingface/tokenizers)
- [YouTokenToMe](https://github.com/VKCOM/YouTokenToMe)

Их сравнивают [здесь](https://github.com/VKCOM/YouTokenToMe/blob/master/benchmark.md) или [здесь](https://towardsdatascience.com/a-small-timing-experiment-on-the-new-tokenizers-library-a-write-up-7caab6f80ea6). Также специальные токенайзеры, которые специализируются на "незападные" языки. Но не будем на них останавливаться.

Теперь ты знаешь достаточно, чтобы написать обработчик данных. Что надо сделать: получить из данных предложения, закодировать их, получить аттенш маску и тип токенов, не забыть про таргет. 

P.S. Есть более быстрая версия токенизатора для BERT внутри `transformers`, `BertTokenizerFast`. 

P.S.S. Теперь надо использовать только функционал токенайзера для кодирования предложений, без велосипедов.

In [0]:
class TextClassificationDataset(Dataset):
    def __init__(self, data, tokenizer, b=False):
        self.data = data
        self.tokenizer = tokenizer

        if b:
          self.encoded_plus = [tokenizer.encode_plus(data['question1'].iloc[i], text_pair = data['question2'].iloc[i], max_length=max_seq_length, pad_to_max_length=True) for i in range(len(data))]

        else:
          self.encoded_plus = [tokenizer.encode_plus(data['sentence'].iloc[i], max_length=max_seq_length, pad_to_max_length=True) for i in range(len(data))]

        # <YOUR CODE>
        self.input_ids = np.array([i['input_ids'] for i in self.encoded_plus])
        self.attention_mask = np.array([i['attention_mask'] for i in self.encoded_plus])
        self.token_type_ids = np.array([i['token_type_ids'] for i in self.encoded_plus])
        if b:
          self.targets = data['is_duplicate'].values
        else:
          self.targets = data['label'].values
        # </YOUR CODE>

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'token_type_ids': self.token_type_ids[idx],
            'targets': int(self.targets[idx])
        }

In [0]:
pretrained_model_name = "google/bert_uncased_L-6_H-256_A-4"
tokenizer = transformers.BertTokenizer.from_pretrained(pretrained_model_name)

Воспользуйтесь семинаром и построй модель для классификации предложений.

(Подсказка: весь код BERT-а из семинара доступен из библиотеки `transformers`)

In [0]:
# models to try
# "google/bert_uncased_L-2_H-128_A-2" "google/bert_uncased_L-4_H-128_A-2 " "google/bert_uncased_L-2_H-256_A-4 "
# 'google/bert_uncased_L-4_H-256_A-4' 'google/bert_uncased_L-6_H-256_A-4 

In [0]:
from transformers import BertModel as bertik

In [0]:
model = bert.from_pretrained(pretrained_model_name)

In [0]:
class BertForSequenceClassification(nn.Module):
    def __init__(self, pretrained_model_name: str, num_labels: int, dropout=0.25):
        super().__init__()

        # <YOUR CODE>
        self.bert = bertik.from_pretrained(pretrained_model_name)

        self.classifier = nn.Linear(self.bert.config.to_dict()['hidden_size'], num_labels)
        
        self.dropout = nn.Dropout(dropout)
        # </YOUR CODE>

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
        assert attention_mask is not None, "attention mask is none"
        
        bert_output = self.bert(input_ids=input_ids,
                                attention_mask=attention_mask,
                                token_type_ids=token_type_ids)
       
        #<YOUR CODE>
        hidden_state = bert_output[0]  # (bs, seq_len, dim)
        pooled_output = hidden_state[:, 0]  # (bs, dim)
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        logits = self.classifier(pooled_output)  # (bs, dim)
        
        
        #</YOUR CODE>

        return logits

Выбери из [списка](https://huggingface.co/models?search=google%2Fbert_) несколько моделей, которые ты будешь обучать. Сравни их качество на выбранных датасетах. 

Лучше всего будет выбрать одну основную конфигурацию, и другие с небольшим изменением. Например, пройтись по такой сетке: `{'layers': [2, 4], 'num_heads': [2, 4]}`. 

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# <YOUR CODE>
num_labels = 2
pretrained_model_name = 'google/bert_uncased_L-6_H-256_A-4'#"bert-base-uncased"
tokenizer = transformers.BertTokenizer.from_pretrained(pretrained_model_name)
model = BertForSequenceClassification(pretrained_model_name, num_labels=num_labels)
# </YOUR CODE>

model.to(device)
print("Success!")

In [0]:
from torch.utils.data import DataLoader, RandomSampler, Dataset

In [0]:
batch_size = 32


# <YOUR CODE>
train_dataset = TextClassificationDataset(train_pd,  tokenizer, b=True)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

valid_dataset = TextClassificationDataset(val_pd,  tokenizer, b=True)
valid_sampler = RandomSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=batch_size)

test_dataset = TextClassificationDataset(test_pd,  tokenizer, b=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

dataloaders = {
    "train": train_dataloader,
    "valid": valid_dataloader    
}
# </YOUR CODE>

In [0]:
seed = 404
set_global_seed(seed)
prepare_cudnn(True)

In [0]:
# Гиперпараметры для обучения модели. Подбери нужные для каждой модели.

epochs = 5
lr = 1e-5
warmup_steps = len(train_dataloader) // 2
t_total = len(train_dataloader) * epochs

Добавь Loss, Optimizer и Scheduler.

In [0]:
optimizer_grouped_parameters = [
    {"params": [p for n, p in model.named_parameters()], "weight_decay": 0.0},
]

# <YOUR CODE>
criterion = torch.nn.CrossEntropyLoss()
optimizer = transformers.AdamW(optimizer_grouped_parameters, lr=lr)
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
)
# </YOUR CODE>

In [0]:
log_dir = 'logs/'

Для обучения модели воспользуемся библиотекой `catalyst`.

In [0]:
runner = SupervisedRunner(
    input_key=(
        "input_ids",
        "attention_mask",
        "token_type_ids"
    )
)

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders= dataloaders,
    callbacks=[
        AccuracyCallback(num_classes=num_labels),
        SchedulerCallback(mode='batch'),
        
    ],
    logdir=log_dir,
    num_epochs= epochs,
    verbose=True,
)

# SST-2

In [0]:
from sklearn.metrics import accuracy_score, f1_score

In [0]:
# function to get a test prediction
y_pred = []
for batch in test_dataloader:
  logits = runner.predict_batch(batch)
  
  y_pred.extend(logits['logits'].cpu().argmax(axis=1).tolist())

google/bert_uncased_L-2_H-128_A-2

In [0]:
print('Test accuracy:',accuracy_score(test_dataset.targets, y_pred)*100, "%") #0.8096330275229358
print('Test f1:',f1_score(test_dataset.targets, y_pred)*100, "%") #0.8139013452914797

Test accuracy: 80.96330275229357 %
Test f1: 81.39013452914797 %


google/bert_uncased_L-4_H-128_A-2 

In [0]:
print('Test accuracy:',accuracy_score(test_dataset.targets, y_pred)*100, "%") #79.81651376146789 %
print('Test f1:',f1_score(test_dataset.targets, y_pred)*100, "%") #80.22471910112358 %

Test accuracy: 79.81651376146789 %
Test f1: 80.22471910112358 %


google/bert_uncased_L-2_H-256_A-4

In [0]:
print('Test accuracy:',accuracy_score(test_dataset.targets, y_pred)*100, "%") #82.11009174311926
print('Test f1:',f1_score(test_dataset.targets, y_pred)*100, "%") #82.47191011235955 %

Test accuracy: 82.11009174311926 %
Test f1: 82.47191011235955 %


google/bert_uncased_L-4_H-256_A-4

In [0]:
print('Test accuracy:',accuracy_score(test_dataset.targets, y_pred)*100, "%") #84.17431192660551 %
print('Test f1:',f1_score(test_dataset.targets, y_pred)*100, "%") #84.56375838926175 %

Test accuracy: 84.17431192660551 %
Test f1: 84.56375838926175 %


google/bert_uncased_L-6_H-256_A-4

In [0]:
print('Test accuracy:',accuracy_score(test_dataset.targets, y_pred)*100, "%") #86.0091743119266
print('Test f1:',f1_score(test_dataset.targets, y_pred)*100, "%") #86.5934065934066 %

Test accuracy: 86.0091743119266 %
Test f1: 86.5934065934066 %


QQP

google/bert_uncased_L-2_H-128_A-2

In [0]:
print('Test accuracy:',accuracy_score(test_dataset.targets, y_pred)*100, "%") #79.80728740927894 %
print('Test f1:',f1_score(test_dataset.targets, y_pred)*100, "%") #76.36827458256029 %

Test accuracy: 79.80728740927894 %
Test f1: 76.36827458256029 %


google/bert_uncased_L-4_H-128_A-2

In [0]:
print('Test accuracy:',accuracy_score(test_dataset.targets, y_pred)*100, "%") #82.76981001213743 %
print('Test f1:',f1_score(test_dataset.targets, y_pred)*100, "%") #78.5599802737024 %

Test accuracy: 82.76981001213743 %
Test f1: 78.5599802737024 %


google/bert_uncased_L-2_H-256_A-4

In [0]:
print('Test accuracy:',accuracy_score(test_dataset.targets, y_pred)*100, "%") #83.94392014069506
print('Test f1:',f1_score(test_dataset.targets, y_pred)*100, "%") #79.35011150047787 

Test accuracy: 83.94392014069506 %
Test f1: 79.35011150047787 %


google/bert_uncased_L-4_H-256_A-4

In [0]:
print('Test accuracy:',accuracy_score(test_dataset.targets, y_pred)*100, "%") #85.62086646355057 %
print('Test f1:',f1_score(test_dataset.targets, y_pred)*100, "%") #81.56967330221924 %

Test accuracy: 85.62086646355057 %
Test f1: 81.56967330221924 %


google/bert_uncased_L-6_H-256_A-4

In [0]:
print('Test accuracy:',accuracy_score(test_dataset.targets, y_pred)*100, "%") #86.00975948081543 %
print('Test f1:',f1_score(test_dataset.targets, y_pred)*100, "%") #81.66233766233766 %

Test accuracy: 86.00975948081543 %
Test f1: 81.66233766233766 %


In [0]:
res = pd.DataFrame(list(zip([80.96, 79.81, 82.11, 84.17, 86.01], [79.81, 82.77, 83.94, 85.62, 86.01, 81.66],
[81.39, 80.22, 82.47, 84.56, 86.59], [76.37, 78.56, 79.36, 81.57, 81.66])), columns = ['SST-2 acc', 'QQP acc', 'SST-2 f1', 'QQP f1'], index=['google/bert_uncased_L-2_H-128_A-2', 'google/bert_uncased_L-4_H-128_A-2', 'google/bert_uncased_L-2_H-256_A-4','google/bert_uncased_L-4_H-256_A-4', 'google/bert_uncased_L-6_H-256_A-4'])                                                                 

Ииии отчет!

Напиши внизу небольшой отчет о проделанной работе. Ожидается сравнение результатов модели с разным количеством голов/слоев на разных датасетах на `test`. Если для оценки качества на датасете используется необычная метрика(не Accuracy или F1), то можно использовать один из них. Было бы круто, если бы вычислялась нужная метрика и она использовалась в отчете.

In [0]:
res

,SST-2 acc,QQP acc,SST-2 f1,QQP f1
google/bert_uncased_L-2_H-128_A-2,80.96,79.81,81.39,76.37
google/bert_uncased_L-4_H-128_A-2,79.81,82.77,80.22,78.56
google/bert_uncased_L-2_H-256_A-4,82.11,83.94,82.47,79.36
google/bert_uncased_L-4_H-256_A-4,84.17,85.62,84.56,81.57
google/bert_uncased_L-6_H-256_A-4,86.01,86.01,86.59,81.66


 Итак, все результаты эксериментов представлены в таблице res. Для сравнения моделей были использованы датасеты SST-2 и QQP. В первом случае задача сентимент классификации (positive/negative) с соответствующими лейблами 1/0, во втором случае также бинарная классификация, но уже для пары предложений нужно определить, являются ли они дубликатами по значению или же нет (1/0). В качестве метрик использованы классические для классификации accuracy и f1. Все модели обучались 5 эпох с шагом 1e-5. и. scheduler. Использованные модели: google/bert_uncased_L-2_H-128_A-2, google/bert_uncased_L-4_H-128_A-2, google/bert_uncased_L-2_H-256_A-4, google/bert_uncased_L-4_H-256_A-4, google/bert_uncased_L-6_H-256_A-4	8. Таким образом, рассматривались модели с [2, 4, 6] слоями, [128, 256] размерностью вектора скрытого состояния и [2, 4] аттеншн головами.  По таблице отчетливо прослеживается, что чем более мощная и сложная модель - тем лучших результатов в средем она достигает. Хотя есть и  исключение,  Берт с 4 слоями и hidden 128 уступает аналогичному Берту с 2 слоями по accuracy на обеих задачах. Однако во всех остальных случаях данная закономерность не нарушается и самые высокие показатели для обоих датасетов по обеим метрикам продемонстрировала модель google/bert_uncased_L-6_H-256_A-4, которая имеет целых 6 слоев, hidden 256 и 4 головы аттешна.